<a href="https://colab.research.google.com/github/Soham0410/Sentimental-Analysis/blob/main/Recurrent_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow
!pip install kaggle

In [ ]:
# Configuring the path of Kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Downloading the dataset
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 95% 77.0M/80.9M [00:00<00:00, 94.1MB/s]
100% 80.9M/80.9M [00:00<00:00, 92.6MB/s]


In [ ]:
from zipfile import ZipFile
dataset = 'sentiment140.zip'
with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, f1_score
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

In [ ]:
# Download stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Data processing
socialmedia_data = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1')

In [ ]:
# Naming columns
column_names = ['target', 'id', 'date', 'flag', 'user', 'text']
socialmedia_data = pd.read_csv('training.1600000.processed.noemoticon.csv', names=column_names, encoding='ISO-8859-1')

In [ ]:
# Replace target values
socialmedia_data.replace({'target': {4: 1}}, inplace=True)

In [ ]:
# Initialize PorterStemmer
port_stem = PorterStemmer()

In [ ]:
# Stemming function
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [ ]:
# Apply stemming
socialmedia_data['stemmed_content'] = socialmedia_data['text'].apply(stemming)

In [ ]:
# Separating the data and label
X = socialmedia_data['stemmed_content']
Y = socialmedia_data['target'].values

In [ ]:
# Train-test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [ ]:
# Tokenization and padding
max_words = 10000
max_len = 100

In [ ]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

In [ ]:
import pickle

In [ ]:
# Save the tokenizer
with open('tokenizer.pkl', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [ ]:
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

In [ ]:
# Building the RNN model
model = Sequential()
model.add(Embedding(max_words, 128, input_length=max_len))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(X_train_pad, Y_train, epochs=3, batch_size=128, validation_split=0.2)

Epoch 1/3
8000/8000 [==============================] - 5136s 641ms/step - loss: 0.4785 - accuracy: 0.7698 - val_loss: 0.4601 - val_accuracy: 0.7807
Epoch 2/3
8000/8000 [==============================] - 5249s 656ms/step - loss: 0.4525 - accuracy: 0.7862 - val_loss: 0.4542 - val_accuracy: 0.7843
Epoch 3/3
8000/8000 [==============================] - 5255s 657ms/step - loss: 0.4384 - accuracy: 0.7945 - val_loss: 0.4547 - val_accuracy: 0.7837


In [ ]:
# Evaluate the model on train data
from sklearn.metrics import recall_score
Y_train_pred_prob = model.predict(X_train_pad)
Y_train_pred = (Y_train_pred_prob > 0.5).astype("int32")

train_accuracy = accuracy_score(Y_train, Y_train_pred)
train_precision = precision_score(Y_train, Y_train_pred)
train_recall = recall_score(Y_train, Y_train_pred)
train_f1 = f1_score(Y_train, Y_train_pred)

print(f'Accuracy Score on the training data: {train_accuracy}')
print(f'Precision Score on the training data: {train_precision}')
print(f'Recall Score on the training data: {train_recall}')
print(f'F1 Score on the training data: {train_f1}')

40000/40000 [==============================] - 3492s 87ms/step


In [ ]:
# Evaluate the model on  test data
Y_pred_prob = model.predict(X_test_pad)
Y_pred = (Y_pred_prob > 0.5).astype("int32")

10000/10000 [==============================] - 916s 92ms/step


In [ ]:
accuracy = accuracy_score(Y_test, Y_pred)
precision = precision_score(Y_test, Y_pred)
recall = recall_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred)

print(f'Accuracy Score on the test data: {accuracy}')
print(f'Precision Score on the test data: {precision}')
print(f'Recall Score on the test data: {recall}')
print(f'F1 Score on the test data: {f1}')

Accuracy Score on the test data: 0.784121875
Precision Score on the test data: 0.7664449198487824
Recall Score on the test data: 0.81729375
F1 Score on the test data: 0.7910530375209835


In [ ]:
# Saving the trained model
model.save('trained_rnn_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Load and use the trained model
loaded_model = tf.keras.models.load_model('trained_rnn_model.h5')

In [ ]:
# Make predictions
def predict_sentiment(text):
    text = stemming(text)
    seq = tokenizer.texts_to_sequences([text])
    pad = pad_sequences(seq, maxlen=max_len)
    pred = loaded_model.predict(pad)
    return 'Positive Tweet' if pred[0][0] > 0.5 else 'Negative Tweet'

In [ ]:
# Example predictions
print(predict_sentiment(X_test.iloc[1000]))
print(predict_sentiment(X_test.iloc[3000]))

1/1 [==============================] - 2s 2s/step
Negative Tweet
1/1 [==============================] - 0s 89ms/step
Negative Tweet
